<a href="https://colab.research.google.com/github/aekaba/Customer-Churn-Prediction-with-AI/blob/TAM-1/CustomerChurnAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Abonelik Sistemlerinde Müşteri Kaybı Analizi

## Dosya Okuma
  datasetimizi drive a kaydetip oradan okuma yapacağız, Train ve Test olarak 2 tane sınıfımız olduğu için bunları birleştireceğiz.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd

# Read CSV files into dataframes
dfTesting = pd.read_csv('/content/drive/MyDrive/Customer Churn Dataset/customer_churn_dataset-testing-master.csv')
dfTraining = pd.read_csv('/content/drive/MyDrive/Customer Churn Dataset/customer_churn_dataset-training-master.csv')

mergedDF = pd.merge(dfTesting, dfTraining, how='outer')

df = pd.DataFrame(mergedDF)

<ipython-input-4-dfbb092143bd>:7: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  mergedDF = pd.merge(dfTesting, dfTraining, how='outer')


505207

In [5]:
print("Toplam Satır Sayısı: ",len(df))
df.head()

Toplam Satır Sayısı:  505207


,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,1.0,22.0,Female,25.0,14.0,4.0,27.0,Basic,Monthly,598.0,9.0,1.0
1,2.0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.0,17.0,1.0
2,2.0,41.0,Female,28.0,28.0,7.0,13.0,Standard,Monthly,584.0,20.0,0.0
3,3.0,47.0,Male,27.0,10.0,2.0,29.0,Premium,Annual,757.0,21.0,0.0
4,3.0,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.0,6.0,1.0
